### Code

In [1]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell
#mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import numpy as np # Included
import pandas as pd
import csv
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer as DV

#np.set_printoptions(threshold=sys.maxsize)

dir1="/home/ubuntu/dataset/"
#dir1="/Users/ejl334/Downloads/IoT Dataset by UC/Concatenated/Archive/"

from numpy import genfromtxt
benign_traffic = genfromtxt(dir1+'benign_traffic.csv', delimiter=',')
malicious_traffic_dataset = genfromtxt(dir1+'mirai_udp.csv', delimiter=',')

/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
# DELETE COLUMNS WITH NAN
print("========================================================================")
print("***BENIGN TRAFFIC***")
print("========================================================================")
print("***DIMENSIONS***")
numrows = len(benign_traffic)
numcols = len(benign_traffic[0])
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

print("***NAN ROWS***")
nan_rows=np.argwhere(np.isnan(benign_traffic).any(axis=1))
print(nan_rows)

print("***** NAN Removal *****")
benign_traffic = benign_traffic[~np.isnan(benign_traffic).any(axis=1)]

print("***DIMENSIONS***")
numrows_good = len(benign_traffic)
numcols = len(benign_traffic[0])
print("Dimensions Rows: %s Columns: %s" %(numrows_good,numcols))

print("***NAN ROWS***")
nan_rows=np.argwhere(np.isnan(benign_traffic).any(axis=1))
print(nan_rows)

print("========================================================================")
print("***MALICIOUS TRAFFIC***")
print("========================================================================")
print("***DIMENSIONS ML***")
malicious_dataset = malicious_traffic_dataset

numrows_ml = len(malicious_dataset)
numcols_ml = len(malicious_dataset[0])
print("Dimensions ML Rows: %s Columns: %s" %(numrows_ml,numcols_ml))

print("***NAN ROWS ML***")
nan_rows_ml=np.argwhere(np.isnan(malicious_dataset).any(axis=1))
print(nan_rows_ml)

print("***** NAN Removal ML *****")
malicious_dataset = malicious_dataset[~np.isnan(malicious_dataset).any(axis=1)]

print("***DIMENSIONS ML***")
numrows_bad = len(malicious_dataset)
numcols_bad = len(malicious_dataset[0])
print("Dimensions ML Rows: %s Columns: %s" %(numrows_bad,numcols_bad))
print("***NAN ROWS ML***")
nan_rows_ml=np.argwhere(np.isnan(malicious_dataset).any(axis=1))
print(nan_rows_ml)

***BENIGN TRAFFIC***
***DIMENSIONS***
Dimensions Rows: 555941 Columns: 115
***NAN ROWS***
[[     0]
 [ 49549]
 [ 62663]
 [101764]
 [277005]
 [339160]
 [437675]
 [489826]
 [536412]]
***** NAN Removal *****
***DIMENSIONS***
Dimensions Rows: 555932 Columns: 115
***NAN ROWS***
[]
***MALICIOUS TRAFFIC***
***DIMENSIONS ML***
Dimensions ML Rows: 1230006 Columns: 115
***NAN ROWS ML***
[[      0]
 [ 237666]
 [ 389148]
 [ 606183]
 [ 762432]
 [ 921041]
 [1072921]]
***** NAN Removal ML *****
***DIMENSIONS ML***
Dimensions ML Rows: 1229999 Columns: 115
***NAN ROWS ML***
[]


In [3]:
benign_traffic[0]

array([1.00000000e+00, 6.00000000e+01, 0.00000000e+00, 1.00000000e+00,
       6.00000000e+01, 0.00000000e+00, 1.00000000e+00, 6.00000000e+01,
       0.00000000e+00, 1.00000000e+00, 6.00000000e+01, 0.00000000e+00,
       1.00000000e+00, 6.00000000e+01, 0.00000000e+00, 1.00000000e+00,
       6.00000000e+01, 0.00000000e+00, 1.00000000e+00, 6.00000000e+01,
       0.00000000e+00, 1.00000000e+00, 6.00000000e+01, 0.00000000e+00,
       1.00000000e+00, 6.00000000e+01, 0.00000000e+00, 1.00000000e+00,
       6.00000000e+01, 0.00000000e+00, 1.00000000e+00, 6.00000000e+01,
       0.00000000e+00, 6.00000000e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 6.00000000e+01, 0.00000000e+00,
       6.00000000e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 6.00000000e+01, 0.00000000e+00, 6.00000000e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       6.00000000e+01, 0.00000000e+00, 6.00000000e+01, 0.00000000e+00,
      

In [4]:
# DELETE UNWANTED COLUMNS
benign_traffic = np.delete(benign_traffic, [2,5,8,11,14,17,20,23,26,29,\
                                            34,35,41,42,48,49,55,56,62,63,\
                                            66,67,69,70,72,73,75,76,78,79,\
                                            84,85,91,92,98,99,105,106,112,113,], axis=1)
malicious_dataset = np.delete(malicious_dataset, [2,5,8,11,14,17,20,23,26,29,\
                                            34,35,41,42,48,49,55,56,62,63,\
                                            66,67,69,70,72,73,75,76,78,79,\
                                            84,85,91,92,98,99,105,106,112,113,], axis=1)


In [5]:
print("***DIMENSIONS BENIGN***")
numrows_good = len(benign_traffic)
numcols_good = len(benign_traffic[0])
print("Dimensions Rows: %s Columns: %s" %(numrows_good,numcols_good))
print("***DIMENSIONS MALICIOUS***")
numrows_bad = len(malicious_dataset)
numcols_bad = len(malicious_dataset[0])
print("Dimensions Rows: %s Columns: %s" %(numrows_bad,numcols_bad))

***DIMENSIONS BENIGN***
Dimensions Rows: 555932 Columns: 75
***DIMENSIONS MALICIOUS***
Dimensions Rows: 1229999 Columns: 75


In [6]:
# Define Labels
y_benings = np.zeros(len(benign_traffic),dtype=int)
y_malicious = np.ones(len(malicious_dataset),dtype=int)

complete_dataset = np.concatenate((benign_traffic, malicious_dataset), 0)
print("*** COMPLETE DATASET ***")
numrows = len(complete_dataset)
numcols = len(complete_dataset[0])
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))
complete_dataset_labels = np.concatenate((y_benings,y_malicious),0)
print("*** COMPLETE DATASET Labels ***")
numrows = len(complete_dataset_labels)
numcols = len(complete_dataset_labels)
print("Dimensions Rows: %s" %(numrows))

*** COMPLETE DATASET ***
Dimensions Rows: 1785931 Columns: 75
*** COMPLETE DATASET Labels ***
Dimensions Rows: 1785931


In [7]:
# NORMALIZE DATASET
scaler = MinMaxScaler(feature_range=(0, 1))
dataX_ = scaler.fit_transform(complete_dataset)

In [9]:
# Vectorize Data
X_=dataX_
vector_train_x = X_.reshape((len(X_), 1, 75))
print(vector_train_x.shape)

(1785931, 1, 75)


# LSTM MODEL

In [11]:
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection

seed=7
np.random.seed(seed)
cvscores = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
print(vector_train_x.shape)
for index, (train, test) in enumerate(kfold.split(vector_train_x, complete_dataset_labels)):
    
    print("Training on fold " + str(index+1) + "/10...")
    
    batch_size = 250000
    model = Sequential()
    model.add(LSTM(28, input_shape=(1,75), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(28))
    model.add(Dropout(0.5))
    model.add(Dense(28))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
    model.summary()
    model.fit(vector_train_x[train], complete_dataset_labels[train], epochs=20, batch_size=batch_size, verbose=2, shuffle=True,callbacks=[early_stopping]) #validation_data=(vector_validation_x, Y_validation)
    scores = model.evaluate(vector_train_x[test], complete_dataset_labels[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
    
    

(1785931, 1, 75)
Training on fold 1/10...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1, 28)             11648     
_________________________________________________________________
dropout_5 (Dropout)          (None, 1, 28)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 28)                6384      
_________________________________________________________________
dropout_6 (Dropout)          (None, 28)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 28)                812       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 29        
Total params: 18,873
Trainable params: 18,873
Non-trainable params: 0
________

/home/ubuntu/.local/lib/python3.5/site-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 4s - loss: 0.2372 - acc: 0.6887
Epoch 3/20
 - 4s - loss: 0.2229 - acc: 0.6887
Epoch 4/20
 - 4s - loss: 0.2043 - acc: 0.6887
Epoch 5/20
 - 4s - loss: 0.1836 - acc: 0.6887
Epoch 6/20
 - 4s - loss: 0.1651 - acc: 0.6887
Epoch 7/20
 - 4s - loss: 0.1493 - acc: 0.6887
Epoch 8/20
 - 4s - loss: 0.1322 - acc: 0.6888
Epoch 9/20
 - 4s - loss: 0.1116 - acc: 0.7135
Epoch 10/20
 - 4s - loss: 0.0881 - acc: 0.9051
Epoch 11/20
 - 4s - loss: 0.0637 - acc: 0.9804
Epoch 12/20
 - 4s - loss: 0.0417 - acc: 0.9988
Epoch 13/20
 - 4s - loss: 0.0254 - acc: 0.9995
Epoch 14/20
 - 4s - loss: 0.0151 - acc: 0.9995
Epoch 15/20
 - 4s - loss: 0.0093 - acc: 0.9996
Epoch 16/20
 - 4s - loss: 0.0060 - acc: 0.9996
Epoch 17/20
 - 4s - loss: 0.0043 - acc: 0.9996
Epoch 18/20
 - 4s - loss: 0.0032 - acc: 0.9997
Epoch 19/20
 - 4s - loss: 0.0026 - acc: 0.9997
Epoch 20/20
 - 5s - loss: 0.0021 - acc: 0.9997
acc: 99.98%
Training on fold 2/10...
Model: "sequential_4"
_________________________________________________________________
L

/home/ubuntu/.local/lib/python3.5/site-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 4s - loss: 0.2265 - acc: 0.6887
Epoch 3/20
 - 4s - loss: 0.2071 - acc: 0.6887
Epoch 4/20
 - 4s - loss: 0.1845 - acc: 0.6887
Epoch 5/20
 - 4s - loss: 0.1625 - acc: 0.6887
Epoch 6/20
 - 4s - loss: 0.1433 - acc: 0.6887
Epoch 7/20
 - 4s - loss: 0.1245 - acc: 0.6894
Epoch 8/20
 - 4s - loss: 0.1033 - acc: 0.7670
Epoch 9/20
 - 4s - loss: 0.0797 - acc: 0.9511
Epoch 10/20
 - 4s - loss: 0.0564 - acc: 0.9894
Epoch 11/20
 - 4s - loss: 0.0367 - acc: 0.9994
Epoch 12/20
 - 4s - loss: 0.0227 - acc: 0.9995
Epoch 13/20
 - 4s - loss: 0.0139 - acc: 0.9995
Epoch 14/20
 - 4s - loss: 0.0088 - acc: 0.9996
Epoch 15/20
 - 4s - loss: 0.0059 - acc: 0.9996
Epoch 16/20
 - 4s - loss: 0.0042 - acc: 0.9996
Epoch 17/20
 - 4s - loss: 0.0032 - acc: 0.9997
Epoch 18/20
 - 4s - loss: 0.0026 - acc: 0.9997
Epoch 19/20
 - 4s - loss: 0.0022 - acc: 0.9997
Epoch 20/20
 - 4s - loss: 0.0018 - acc: 0.9997
acc: 99.98%
Training on fold 3/10...
Model: "sequential_5"
_________________________________________________________________
L

...

/home/ubuntu/.local/lib/python3.5/site-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 4s - loss: 0.2333 - acc: 0.6887
Epoch 3/20
 - 4s - loss: 0.2163 - acc: 0.6887
Epoch 4/20
 - 4s - loss: 0.1955 - acc: 0.6887
Epoch 5/20
 - 4s - loss: 0.1752 - acc: 0.6887
Epoch 6/20
 - 4s - loss: 0.1586 - acc: 0.6887
Epoch 7/20
 - 4s - loss: 0.1423 - acc: 0.6887
Epoch 8/20
 - 4s - loss: 0.1218 - acc: 0.6947
Epoch 9/20
 - 4s - loss: 0.0971 - acc: 0.8341
Epoch 10/20
 - 4s - loss: 0.0704 - acc: 0.9716
Epoch 11/20
 - 4s - loss: 0.0455 - acc: 0.9979
Epoch 12/20
 - 4s - loss: 0.0266 - acc: 0.9993
Epoch 13/20
 - 4s - loss: 0.0151 - acc: 0.9993
Epoch 14/20
 - 4s - loss: 0.0088 - acc: 0.9994
Epoch 15/20
 - 4s - loss: 0.0056 - acc: 0.9995
Epoch 16/20
 - 4s - loss: 0.0039 - acc: 0.9996
Epoch 17/20
 - 4s - loss: 0.0029 - acc: 0.9996
Epoch 18/20
 - 4s - loss: 0.0023 - acc: 0.9997
Epoch 19/20
 - 4s - loss: 0.0019 - acc: 0.9997
Epoch 20/20
 - 4s - loss: 0.0016 - acc: 0.9998
acc: 99.98%
Training on fold 5/10...
Model: "sequential_7"
_________________________________________________________________
L

/home/ubuntu/.local/lib/python3.5/site-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 4s - loss: 0.2363 - acc: 0.6887
Epoch 3/20
 - 4s - loss: 0.2220 - acc: 0.6887
Epoch 4/20
 - 4s - loss: 0.2037 - acc: 0.6887
Epoch 5/20
 - 4s - loss: 0.1832 - acc: 0.6887
Epoch 6/20
 - 4s - loss: 0.1645 - acc: 0.6887
Epoch 7/20
 - 4s - loss: 0.1483 - acc: 0.6887
Epoch 8/20
 - 4s - loss: 0.1312 - acc: 0.6893
Epoch 9/20


KeyboardInterrupt: 

## model.predict()